In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df2 = pd.read_csv('last12months/12months/220330932.csv')
df2['dt'] = pd.to_datetime(df2['TIMESTAMP'],unit='ms')


/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (21,23,25,27,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def gotAnomalies(df:pd.DataFrame):
    unique = df['Anomalia #1'].unique()
    b = False
    for v in unique:
        if v == 0 or v == np.nan:
            b = False
        else:
            b = True
            break
    return b, unique


In [4]:
print(gotAnomalies(df2))

(True, array([nan,  0., 76.]))


In [5]:
def cleanCSV(df):
  cols_to_check = df.columns
  arr_col = []
  for col in df.columns:
    if df[col].isna().all():
        print("La colonna", col, "contiene solo valori NaN.")
        arr_col.append(col)
  df_cleaned = df.drop(columns=arr_col)
  df_cleaned = df_cleaned.replace('---', 0)
  return df_cleaned

In [6]:
def sortDf(df):
    if 'TIMESTAMP' in df.columns:
        df = df.sort_values('TIMESTAMP',ascending=True)
        df.index = range(len(df))
    else:
        print("La colonna TIMESTAMP non è presente nel dataframe.")
        return None
    return df

In [7]:
def resampleDf(df,datetime):
    if 'dt' in df.columns:
        df.set_index('dt', inplace=True)
    df_noNan = df.ffill()
    df_resampled = df_noNan.resample(datetime).ffill()
    df_resampled = df_resampled.replace('---', 0)
    return df_resampled
    

In [8]:
def checkFileInFolder(folder, file):
    if file in os.listdir(folder):
        return True
    return False

In [9]:
def getResampledDf(df,file_path):
    if not checkFileInFolder(folder ='last12months/12months',file=file_path):
        df_sorted = sortDf(df)
        df_sorted = cleanCSV(df_sorted)
        df_resampled = resampleDf(df_sorted,'30s')
        df_resampled.to_csv('last12months/12months/' + file_path)
        return df_sorted, df_resampled
    else:
        df_resampled = pd.read_csv('last12months/12months/'+ file_path)
        return None, df_resampled

    

In [10]:
df2_sorted, df2_resampled = getResampledDf(df2,'220330932_resampled30s.csv')

/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: DtypeWarning: Columns (22,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


In [11]:
df2_resampled['Anomalia #1'] = df2_resampled['Anomalia #1'].replace(np.nan,0)
df2_resampled.head()

,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
0,2022-06-30 22:00:00,1656626400000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-30 22:00:30,1656626400000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-30 22:01:00,1656626400000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-30 22:01:30,1656626400000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-30 22:02:00,1656626400000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def getUniques(df):
    cols = df.columns
    dict_unique = dict.fromkeys(cols,None)
    for col in cols:
        dict_unique[col] = df[col].unique()
    return dict_unique

In [13]:
def getBooelan(dict):
    arr = []
    for key, value in dict.items():
        if len(value) < 4:
            b = True
            for v in value:
                # print(v,pd.isna(v))
                if pd.isna(v)==False:
                    if int(v) != 0 and int(v) != 1:
                        # print('ciao')
                        b = False
                        break
            # print(value,b)
            if b:
                arr.append(key)
    return arr

In [14]:
def printUniques(dict,tp):
    booleans = getBooelan(dict)
    for key, value in dict.items():
        counts = []
        if key in booleans and tp ==1:
            print(key, value)
        elif tp==2 and len(value) > 4 :
            print(key, value[0:5])

In [15]:
def countOccurences(df,dict):
    booleans = getBooelan(dict)
    counters = dict.fromkeys(booleans,None)
    for key, value in dict.items():
        if key in booleans:
            value = value[1:]
            counts = dict.fromkeys(value,None)
            for v in value:
                if not pd.isna(v):
                    counts[v]= df[key].value_counts()[v]
            counters[key] = counts
            # print(key, counts, max(counts))
    return counters
            

In [16]:
def getMax(dict):
    maxs = dict.fromkeys(dict.keys(),None)
    for key, value in dict.items():
        # print(key, value)
        for k, v in value.items():
            if v == max(value.values()):
                maxs[key] = k
    return maxs

In [17]:
def plotColumn(df,col):
    if col not in df.columns:
        print("La colonna", col, "non è presente nel dataframe.")
        return None
    df_toPlot = df[col]
    plt.plot(df_toPlot)
    try: 
        plt.ylim(0, max(df_toPlot.unique())+1)
    except:
        pass

    plt.show()


In [18]:
dict_unique2 = getUniques(df2_resampled)
arr_booleani2 = getBooelan(dict_unique2)


In [19]:
def getLastValues(df):
    d_sample = df.iloc[-1]
    dictionary = dict.fromkeys(df.columns,None)

    for col in df.columns:
        if col in d_sample.index:
            dictionary[col] = d_sample[col]

    return dictionary

In [20]:
def reversedMax(dizionario):
    for key, value in dizionario.items():
        if value == 1:
            dizionario[key] = 0
        else:
            dizionario[key] = 1
    return dizionario

In [21]:
def getGeneralBooleans(df2):
    b2 = getBooelan(getUniques(df2))
    intersection = list(set(b2))
    intersection.sort()
    return intersection

In [22]:
def fillBooleans(df,booleans,dizionario):
    for key in booleans:
        if key != 'Modalità Estate/Inverno (solo scrittura)' and key in dizionario.keys():
            print(f'{key}: {dizionario[key]}')        
            df[key] = df[key].replace(np.nan,dizionario[key])
    return df

In [23]:
def fillOthers(df,sample):
    col_s = sample.index
    booleani = getBooelan(getUniques(df))
    for col in df.columns:
        # print(col)
        if col == 'Anomalia #1':
            df[col] = df[col].replace(np.nan,sample[col])
        if col not in booleani and col in col_s:
            # print('in')
            df[col] = df[col].replace(np.nan,sample[col])
    return df

In [24]:
print("Anomalies' values: ",df2_resampled['Anomalia #1'].unique())
num_anomaliesDf2 = df2['Anomalia #1'].value_counts()[76]
print("Number of anomalies type 76 in df2:", num_anomaliesDf2)

Anomalies' values:  [ 0. 76.]
Number of anomalies type 76 in df2: 9


In [26]:
num_anomaliesDf2 = df2_resampled['Anomalia #1'].value_counts()[76]
print("Number of anomalies type 76 in df2:", num_anomaliesDf2)

Number of anomalies type 76 in df2: 1664


In [27]:
column_to_first_valid_index = {}

for column in df2_resampled.columns:
  first_valid_index = df2_resampled[column].first_valid_index()
  column_to_first_valid_index[column] = first_valid_index

#iterate the dict and print the key and value
for key, value in column_to_first_valid_index.items():
  print(key, ' : ', value)

dt  :  0
TIMESTAMP  :  0
Abilitazione al conteggio energia  :  61836
Anomalia #1  :  0
Caldaia  :  48960
Caldaia giornaliero  :  60480
Cloud Status  :  61827
Connection Status  :  61828
Energia parziale in riscaldamento caldaia  :  61874
Energia parziale in sanitario caldaia  :  62076
Energia totale globale  :  61874
Energia totale globale delta  :  62076
Energia totale globale per day  :  60480
Energia totale globale per month  :  0
Energia totale globale per week  :  48960
Energia totale in riscaldamento caldaia  :  61874
Energia totale in sanitario caldaia  :  62076
Fattore di correzione  :  61836
Flame status  :  61833
Flusso sanitario  :  61833
Intercetta della retta della potenza  :  61836
Livello modulazione  :  61833
M64A17E8  :  61874
M64A17E9  :  62076
M64A17EA  :  61874
M64A17EB  :  62076
M64A17EC  :  62076
M64A17ED  :  62076
M64A17EE  :  62076
M64A17EF  :  62076
Media T esterna  :  61833
Modalità Estate/Inverno (solo scrittura)  :  61833
Modo service  :  61836
Modo vacanza 

In [36]:
#DEPRECATE

#iterate the dict and print the min value different from 0 
def getMinValue(dictionary:dict ):
    minus = max(dictionary.values())
    for key,value in dictionary.items():
        # print(key, ':' , value)
        if key == 'TIMESTAMP' or key == 'dt':
            continue
        else:
            if value < minus:
                minus = value
            
                
    return key, minus

def getMaxValue(dictionary:dict ):
    max = 0
    for key,value in dictionary.items():
        # print(key, ':' , value)
        if value > max:
            max = value
            
                
    return key, max


In [ ]:
def 

In [43]:
dfProva = df2_resampled.copy()
dfProva = dfProva.dropna()

In [44]:
dfProva.head()

,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
567147,2023-01-13 20:13:30,1673640808944,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.7,0.0,0.0,0.0,14.9,0.0,0.0,-72.0
567148,2023-01-13 20:14:00,1673640834571,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.6,0.0,0.0,0.0,14.9,0.0,0.0,-72.0
567149,2023-01-13 20:14:30,1673640855914,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.6,0.0,0.0,0.0,14.9,0.0,3529.0,-72.0
567150,2023-01-13 20:15:00,1673640886341,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,23.5,0.0,0.0,0.0,14.8,0.0,3365.0,-43.0
567151,2023-01-13 20:15:30,1673640926632,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,30.8,0.0,0.0,0.0,19.5,0.0,4867.0,-72.0


In [49]:
dfProva.shape

(516942, 57)

## Work with final DataFrame

In [3]:
df = pd.read_csv('last12months/12months/220330932_resampled30s_noNan.csv')

/Users/leonardozecchin/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (22,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
0,2023-01-13 20:13:30,1673640808944,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.7,0.0,0.0,0.0,14.9,0.0,0.0,-72.0
1,2023-01-13 20:14:00,1673640834571,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.6,0.0,0.0,0.0,14.9,0.0,0.0,-72.0
2,2023-01-13 20:14:30,1673640855914,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,14.6,0.0,0.0,0.0,14.9,0.0,3529.0,-72.0
3,2023-01-13 20:15:00,1673640886341,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,23.5,0.0,0.0,0.0,14.8,0.0,3365.0,-43.0
4,2023-01-13 20:15:30,1673640926632,1.0,0.0,80211368.0,12890431.0,2.0,1.0,252.0,236.0,...,0.0,-2.5,30.8,0.0,0.0,0.0,19.5,0.0,4867.0,-72.0


In [7]:
dfAnomalies = df[df['Anomalia #1'] == 76]

In [8]:
dfAnomalies

,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
53620,2023-02-01 11:03:30,1675249407057,1.0,76.0,70625003.0,19062356.0,2.0,1.0,201.0,31.0,...,0.0,-2.5,27.5,0.0,0.0,0.0,23.6,0.0,0.0,-54.0
53621,2023-02-01 11:04:00,1675249433937,1.0,76.0,70625003.0,19062356.0,2.0,1.0,201.0,31.0,...,0.0,-2.5,27.5,0.0,0.0,0.0,23.6,0.0,0.0,-54.0
53622,2023-02-01 11:04:30,1675249462109,1.0,76.0,70625003.0,19062356.0,2.0,1.0,201.0,31.0,...,0.0,-2.5,11.1,0.0,0.0,0.0,17.7,0.0,0.0,-54.0
53623,2023-02-01 11:05:00,1675249491657,1.0,76.0,70625003.0,19062356.0,2.0,1.0,201.0,31.0,...,0.0,-2.5,11.1,0.0,0.0,0.0,17.7,0.0,0.0,-54.0
53624,2023-02-01 11:05:30,1675249517137,1.0,76.0,70625003.0,19062356.0,2.0,1.0,201.0,31.0,...,0.0,-2.5,10.9,0.0,0.0,0.0,15.7,0.0,0.0,-54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55816,2023-02-02 05:21:30,1675315276716,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,17.8,0.0,0.0,0.0,16.4,0.0,0.0,-48.0
55817,2023-02-02 05:22:00,1675315317016,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,17.8,0.0,0.0,0.0,16.4,0.0,0.0,-45.0
55818,2023-02-02 05:22:30,1675315346985,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,17.8,0.0,0.0,0.0,16.4,0.0,0.0,-45.0
55819,2023-02-02 05:23:00,1675315372725,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,17.7,0.0,0.0,0.0,16.4,0.0,0.0,-51.0
